# Data Prepare

This note book is written to guide the data prepare process to download from yahoo finance api and generate data with indicator using C++ compiled executables

In [ ]:

import pandas as pd
import shutil
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
import datetime
## yahoo finance
import yfinance as yf
yf.pdr_override()
import time
## used to executable command
import subprocess

In [ ]:
## ge the time line
start_date=datetime.datetime(2010, 1, 9)
end_date=datetime.datetime(2022, 1, 9)


In [ ]:

## define a list of stocks
stock_list = [
    "^OEX","^NDX","^RUT","^DJI", ## index
]  


for stock in stock_list:
    ## remove the ^ in front of the stock name, otherwise leave the same
    stock_name = stock[1:] if stock[0] == '^' else stock

    ## file location
    file_location = './data/' + stock_name + '.txt'
    ## get yahoo finance from start date to end date
    df = pdr.get_data_yahoo(stock, start=start_date, end=end_date)
    ## drop adjusted close
    df.drop(['Adj Close'], axis=1, inplace=True)
    print(stock)
    print(f'received data from yf for {stock} with {df.shape}')

    df = df.reset_index()
    ## change to date to datetime and format it
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%Y%m%d')
    ## write  it down to csv file because C++ single exe. only takes in flat files
    df.to_csv(file_location, header=None, index=None, sep=' ', mode='a')

    ## use popen to execute the single exe
    ## because the program will require us to key-input to kill the process
    ## we manually kill the program
    
    p=subprocess.Popen(['.\exe\single.exe', file_location, 'VS.txt'])
    time.sleep(4)
    p.kill()
    # # subprocess.call(['.\exe\single.exe', file_location, 'VS.txt'])

    print('enter is input, executable finishing')

    ## single will create file '.\OUTVARS.TXT' 
    ## we move to <ticker_name>_singal.txt file
    ## loaded the file merge with our original data

    signal_location = './data/' + stock_name + '_signal.txt'
    shutil.move('.\OUTVARS.TXT', signal_location)

    indicator = pd.read_csv(signal_location ,delim_whitespace=True)
    indicator['Date'] = pd.to_datetime(indicator['Date'], format = '%Y%m%d')
    indicator['Date'] = indicator['Date'].dt.strftime('%Y%m%d')

    print('indicator file has shape', indicator.shape)

    df_result = df.merge(indicator, left_on = 'Date', right_on = 'Date', how = 'left')
    df_result.to_csv('./data/' + stock_name + '_full.csv', index=False)


